# Working with Twelve Labs in Pixeltable

Twelve Labs provides multimodal embeddings that project text, images, audio, and video into the **same semantic space**. This enables true **cross-modal search** - the most powerful feature of this integration.

**What makes this special?** You can search a video index using *any* modality:

| Query Type | Use Case |
|------------|----------|
| **Text → Video** | "Find clips of a man giving a speech" |
| **Image → Video** | Find videos visually similar to a photo |
| **Audio → Video** | Find videos with similar speech/sounds |
| **Video → Video** | Find videos similar to a clip |

This notebook demonstrates this cross-modal capability with video, then shows how to apply the same embeddings to other modalities.

### Prerequisites
- A Twelve Labs account with an API key ([playground.twelvelabs.io](https://playground.twelvelabs.io/))
- Audio and video must be at least 4 seconds long

## Setup

In [ ]:
%pip install -qU pixeltable twelvelabs

In [ ]:
import os
import getpass

if 'TWELVELABS_API_KEY' not in os.environ:
    os.environ['TWELVELABS_API_KEY'] = getpass.getpass('Enter your Twelve Labs API key: ')

In [ ]:
import pixeltable as pxt
from pixeltable.functions.twelvelabs import embed

# Create a fresh directory for our demo
pxt.drop_dir('twelvelabs_demo', force=True)
pxt.create_dir('twelvelabs_demo')

## Cross-Modal Video Search

Let's index a video and search it using text, images, audio, and other videos - all against the same index.

### Create Video Table and Index

In [ ]:
from pixeltable.functions.video import video_splitter

# Create a table for videos
video_t = pxt.create_table('twelvelabs_demo.videos', {'video': pxt.Video})

# Insert a sample video
video_url = 'https://github.com/pixeltable/pixeltable/raw/main/docs/resources/The-Pursuit-of-Happiness.mp4'
video_t.insert([{'video': video_url}])

In [ ]:
# Create a view that segments the video into searchable chunks
# Twelve Labs requires minimum 4 second segments
video_chunks = pxt.create_view(
    'twelvelabs_demo.video_chunks',
    video_t,
    iterator=video_splitter(
        video=video_t.video,
        duration=5.0,
        min_segment_duration=4.0
    )
)

# Add embedding index for cross-modal search
video_chunks.add_embedding_index(
    'video_segment',
    embedding=embed.using(model_name='marengo3.0')
)

### Text → Video Search

Find video segments matching a text description.

In [ ]:
sim = video_chunks.video_segment.similarity(string="a man with grey hair")

video_chunks.order_by(sim, asc=False).limit(3).select(
    video_chunks.video_segment,
    score=sim
).collect()

### Image → Video Search

Find video segments similar to an image.

In [ ]:
image_query = 'https://github.com/pixeltable/pixeltable/raw/main/docs/resources/The-Pursuit-of-Happiness-Screenshot.png'

sim = video_chunks.video_segment.similarity(image=image_query)

video_chunks.order_by(sim, asc=False).limit(2).select(
    video_chunks.video_segment,
    score=sim
).collect()

### Video → Video Search

Find video segments similar to another video clip.

In [ ]:
video_query = 'https://github.com/pixeltable/pixeltable/raw/main/docs/resources/The-Pursuit-of-Happiness-Video-Extract.mp4'

sim = video_chunks.video_segment.similarity(video=video_query)

video_chunks.order_by(sim, asc=False).limit(2).select(
    video_chunks.video_segment,
    score=sim
).collect()

### Audio → Video Search

Find video segments with similar audio/speech content.

In [ ]:
audio_query = 'https://github.com/pixeltable/pixeltable/raw/main/docs/resources/The-Pursuit-of-Happiness-Audio-Extract.m4a'

sim = video_chunks.video_segment.similarity(audio=audio_query)

video_chunks.order_by(sim, asc=False).limit(2).select(
    video_chunks.video_segment,
    score=sim
).collect()

## Embedding Options

For video embeddings, you can focus on specific aspects:
- `'visual'` - Focus on what you see
- `'audio'` - Focus on what you hear  
- `'transcription'` - Focus on what is said

In [ ]:
# Add a visual-only embedding column
video_chunks.add_computed_column(
    visual_embedding=embed(
        video_chunks.video_segment,
        model_name='marengo3.0',
        embedding_option=['visual']
    )
)

video_chunks.select(
    video_chunks.video_segment,
    video_chunks.visual_embedding
).limit(2).collect()

## Other Modalities: Text, Images, and Documents

Twelve Labs embeddings also work for text, images, and documents. Here's a compact example showing **multiple embedding indexes on a single table**.

In [ ]:
# Create a multimodal content table
content_t = pxt.create_table(
    'twelvelabs_demo.content',
    {
        'title': pxt.String,
        'description': pxt.String,
        'thumbnail': pxt.Image
    }
)

# Add embedding index on text column
content_t.add_embedding_index(
    'description',
    embedding=embed.using(model_name='marengo3.0')
)

# Add embedding index on image column
content_t.add_embedding_index(
    'thumbnail',
    embedding=embed.using(model_name='marengo3.0')
)

In [ ]:
# Insert sample content
content_t.insert([
    {
        'title': 'Beach Sunset',
        'description': 'A beautiful sunset over the ocean with palm trees.',
        'thumbnail': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000025.jpg'
    },
    {
        'title': 'Mountain Hiking',
        'description': 'Hikers climbing a steep mountain trail with scenic views.',
        'thumbnail': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000139.jpg'
    },
    {
        'title': 'City Street',
        'description': 'Busy urban street with cars and pedestrians.',
        'thumbnail': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000042.jpg'
    },
    {
        'title': 'Wildlife Safari',
        'description': 'Elephants and zebras on the African savanna.',
        'thumbnail': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000061.jpg'
    }
])

In [ ]:
# Search by text description
sim = content_t.description.similarity(string="outdoor nature adventure")

content_t.order_by(sim, asc=False).limit(2).select(
    content_t.title,
    content_t.description,
    score=sim
).collect()

In [ ]:
# Search by image similarity
query_image = 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000001.jpg'

sim = content_t.thumbnail.similarity(image=query_image)

content_t.order_by(sim, asc=False).limit(2).select(
    content_t.title,
    content_t.thumbnail,
    score=sim
).collect()

In [ ]:
# Cross-modal: Search images using text!
sim = content_t.thumbnail.similarity(string="animals in nature")

content_t.order_by(sim, asc=False).limit(2).select(
    content_t.title,
    content_t.thumbnail,
    score=sim
).collect()

## Summary

**Twelve Labs + Pixeltable enables:**

- **Cross-modal search**: Query video with text, images, audio, or other videos
- **Multiple indexes per table**: Add embedding indexes on different columns
- **Embedding options**: Focus on visual, audio, or transcription aspects
- **All modalities**: Text, images, audio, video, and documents

### Learn More

- [Twelve Labs Documentation](https://docs.twelvelabs.io/)
- [Pixeltable Documentation](https://docs.pixeltable.com/)